In [1]:
from iroha import IrohaCrypto
from iroha.ed25519 import H
import integration_helpers
from iroha.primitive_pb2 import can_set_my_account_detail
import json
from iroha_helper import *
from new_helper import *
from super_helper import *
from ipfs_functions import *
from loguru import logger

In [2]:
# Index for objects in both user account and project account JSON-LDs.
json_ld_index = 0

# Local path for file upload
directory_path = "upload"

# Directory for file downloads
download_path = "download"

# Read accounts from JSON-LD
user_accounts = read_user_accounts_from_jsonld('datasets/accounts.json')
project_accounts = read_project_accounts_from_jsonld('datasets/projects.json')

#for the index system
index_path = "indexdir"
index = open_dir(index_path)


In [3]:
# Customize the logger format
logger.remove()
logger.add(
    sink=lambda msg: print(msg, end=""),
    format="<green>{time:YYYY-MM-DD HH:mm:ss.SSS}</green> | "
           "<level>{level: <8}</level> | "
           "<cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - "
           "{message}",
    colorize=True,  # Enable colors for supported terminals
)

2

In [4]:
#Manually resets the index on execution
# recreate_index() 

1 - Deploys a smart contract into the Iroha 1 blockchain for details (attributes) setting;

In [5]:
hash = create_detail_contract()
integration_helpers.get_engine_receipts_result(hash)

2025-01-11 22:49:50.104 | INFO     | integration_helpers:tracer:34 - 	Entering "create_detail_contract"
2025-01-11 22:49:50.127 | INFO     | iroha_helper:create_detail_contract:58 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 22:49:50.130 | INFO     | iroha_helper:create_detail_contract:58 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 22:49:51.604 | INFO     | iroha_helper:create_detail_contract:58 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:49:51.634 | INFO     | iroha_helper:create_detail_contract:58 - ('COMMITTED', 5, 0)
2025-01-11 22:49:51.635 | INFO     | integration_helpers:tracer:36 - 	Leaving "create_detail_contract"
2025-01-11 22:49:51.635 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_result"
2025-01-11 22:49:51.653 | INFO     | integration_helpers:get_engine_receipts_result:92 - 
2025-01-11 22:49:51.653 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_result"


2 - Data extraction from JSON-LD.

Extracts account ids from `datasets/accounts.json` and `datasets/projects.json`.

Must update `json_ld_index` with a entry number related to an existing object in `datasets/accounts.json` and `datasets/projects.json`

4 - Sets details for both User and Project accounts providing a logical link between them for later references.

In [6]:
# Example execution of the previous snippet
address = integration_helpers.get_engine_receipts_address(hash)

# Assuming json_ld_index is defined
user_account = user_accounts[json_ld_index]
project_account = project_accounts[json_ld_index]

# Set project_id as a detail for the user account
hash_user_to_project = set_account_detail(
    address, 
    user_account['account_id'], 
    "linked_project", 
    project_account['account_id']
)

# Set user_account_id as a detail for the project account
hash_project_to_user = set_account_detail(
    address, 
    project_account['account_id'], 
    "linked_user", 
    user_account['account_id']
)

# Update the JSON-LD files with the linked details
update_user_account_link(user_account['account_id'], project_account['account_id'])
update_project_account_link(project_account['account_id'], user_account['account_id'])

# Confirming the operation
logger.info(f"User account {user_account['account_id']} linked to project {project_account['account_id']}")
logger.info(f"Project account {project_account['account_id']} linked to user {user_account['account_id']}")

2025-01-11 22:49:51.675 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 22:49:51.694 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 22:49:51.694 | INFO     | iroha_helper:tracer:247 - 	Entering "set_account_detail"
2025-01-11 22:49:51.713 | INFO     | iroha_helper:set_account_detail:299 - None
2025-01-11 22:49:51.715 | INFO     | iroha_helper:set_account_detail:301 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 22:49:51.715 | INFO     | iroha_helper:set_account_detail:301 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 22:49:54.663 | INFO     | iroha_helper:set_account_detail:301 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:49:54.688 | INFO     | iroha_helper:set_account_detail:301 - ('COMMITTED', 5, 0)
2025-01-11 22:49:54.689 | INFO     | iroha_helper:tracer:249 - 	Leaving "set_account_detail"
2025-01-11 22:49:54.689 | INFO     | iroha_helper:tracer:247 - 	Entering "set_acc

3 - Queries Iroha 1 for User account and checks its values

In [7]:
#Query - GetAccountDetail
query = iroha.query('GetAccountDetail',account_id=user_account['account_id'])
# logger.info(query)
IrohaCrypto.sign_query(query, ADMIN_PRIVATE_KEY)
response = net.send_query(query)
# logger.info(response)

user_data = response.account_detail_response
user_details = user_data.detail

logger.info(f'User Account id = {user_account}, {user_details}')

2025-01-11 22:49:57.783 | INFO     | __main__:<module>:11 - User Account id = {'account_id': 'elated_noether@test'}, { "admin@test" : { "linked_project" : "73243@test", "user_json_ld_cid" : "QmS6HCQXDnFaZT4GJmR4Yv2j4vhdqRBdauXZYoazL3KKcf" } }


6 - Queries the user account, locates the project id, queries the project account, gets the metadata and files from IPFS.

In [8]:
# Process the account details response
user_details_dict = json.loads(user_details)  # Convert the string to a JSON object
logger.info(user_details_dict)

# Now you can access the specific key like this
project_id = user_details_dict["admin@test"]["linked_project"]
logger.info(project_id)

2025-01-11 22:49:57.840 | INFO     | __main__:<module>:3 - {'admin@test': {'linked_project': '73243@test', 'user_json_ld_cid': 'QmS6HCQXDnFaZT4GJmR4Yv2j4vhdqRBdauXZYoazL3KKcf'}}
2025-01-11 22:49:57.841 | INFO     | __main__:<module>:7 - 73243@test


In [9]:
schema = get_schema() #super_helper.py

logger.info(schema)

processed_data = process_files(directory_path, project_id, schema) #new_helper.py
 
    

2025-01-11 22:49:57.901 | INFO     | __main__:<module>:3 - <Schema: ['abstract', 'created', 'creator', 'date', 'description', 'file_cid', 'format', 'full_text', 'language', 'metadata_cid', 'modified', 'project_id', 'publisher', 'subject', 'title']>


2025-01-11 22:49:58.067 | INFO     | iroha_helper:tracer:247 - 	Entering "create_contract"
2025-01-11 22:49:58.080 | INFO     | iroha_helper:create_contract:266 - ('STATELESS_VALIDATION_SUCCESS', 1, 0)
2025-01-11 22:49:58.082 | INFO     | iroha_helper:create_contract:266 - ('ENOUGH_SIGNATURES_COLLECTED', 9, 0)
2025-01-11 22:50:00.762 | INFO     | iroha_helper:create_contract:266 - ('STATEFUL_VALIDATION_SUCCESS', 3, 0)
2025-01-11 22:50:00.790 | INFO     | iroha_helper:create_contract:266 - ('COMMITTED', 5, 0)
2025-01-11 22:50:00.791 | INFO     | iroha_helper:tracer:249 - 	Leaving "create_contract"
2025-01-11 22:50:00.791 | INFO     | integration_helpers:tracer:34 - 	Entering "get_engine_receipts_address"
2025-01-11 22:50:00.809 | INFO     | integration_helpers:tracer:36 - 	Leaving "get_engine_receipts_address"
2025-01-11 22:50:00.809 | INFO     | iroha_helper:tracer:247 - 	Entering "set_account_detail"
2025-01-11 22:50:00.822 | INFO     | iroha_helper:set_account_detail:299 - None
2025-

In [10]:
account_detail = get_account_detail(project_id)
logger.info(f"{project_id}, {account_detail}")


2025-01-11 22:50:04.010 | INFO     | __main__:<module>:2 - 73243@test, { "admin@test" : { "file_1" : "QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, QmPiZKhXpmKCJ7scjBNGeU2BxuDEac3AaDrNr55PwPuZZV", "linked_user" : "elated_noether@test", "project_metadata_cid" : "QmW6MDJSfQD8pgMNkqCxNMYzeKkC8YDkCgHtkP5mkKzEqg" } }


In [11]:
# Perform a keyword search
keyword = "must"
search_results, project_ids_with_cids = search_index(index, keyword)

# Check if there are no search results
if not search_results:
    logger.warning(f"No search results found for keyword: '{keyword}'. Exiting the script.")
else:
    # Process each dictionary in search results
    for result_dict in search_results:
        project_id = result_dict.get('project_id')
        file_cid = result_dict.get('file_cid')
        metadata_cid = result_dict.get('metadata_cid')

        if not project_id or not file_cid or not metadata_cid:
            logger.error(f"Missing required data in result: {result_dict}")
            continue

        # Log the retrieved project details
        logger.info(f"Processing Project ID: {project_id}")
        logger.debug(f"File CID: {file_cid}")
        logger.debug(f"Metadata CID: {metadata_cid}")
        # file_metadata_json = download_json_from_ipfs(metadata_cid)
        # logger.info("file_metadata_json:", file_metadata_json)
        

        # Fetch project details from the blockchain
        project_details = get_account_detail(project_id)
        if not project_details:
            logger.error(f"No project details found for Project ID: {project_id}.")
            continue

        logger.debug(f"Fetched project details for {project_id}: {project_details}")

        # Parse blockchain data
        try:
            blockchain_data = json.loads(project_details)
        except json.JSONDecodeError as e:
            logger.error(f"Error decoding project details JSON for {project_id}: {e}")
            continue

        # Validate file CID and fetch project details
        validation_result = fetch_project_details(file_cid, blockchain_data)
        if validation_result["is_valid"]:
            project_metadata_cid = validation_result.get("project_metadata_cid")
            linked_user = validation_result.get("linked_user")
            file_metadata_cid = validation_result.get("metadata_cid")
            
            # download_file(file_metadata_json, download_path, project_id, file_cid)

            logger.info(f"Valid File CID for {project_id}.")
            logger.debug(f"Project Metadata CID: {project_metadata_cid}")
            logger.debug(f"Linked User: {linked_user}")

            # Fetch and process metadata and user details
            if project_metadata_cid:
                logger.debug(f"Processing project metadata CID: {project_metadata_cid}")
                project_metadata = download_json_from_ipfs(project_metadata_cid)
                logger.debug(f"Downloaded project metadata: {project_metadata}")

            if linked_user:
                logger.debug(f"Processing linked user: {linked_user}")
                user_details = get_account_detail(linked_user)
                try:
                    user_details = json.loads(user_details)
                except json.JSONDecodeError as e:
                    logger.error(f"Error decoding user details JSON for {linked_user}: {e}")
                    continue

                user_json_ld_cid = user_details.get("admin@test", {}).get("user_json_ld_cid", None)
                if user_json_ld_cid:
                    logger.info(f"User JSON-LD CID: {user_json_ld_cid}")
                    user_metadata = download_json_from_ipfs(user_json_ld_cid)
                    logger.debug(f"Downloaded user metadata: {user_metadata}")
                else:
                    logger.warning(f"User JSON-LD CID not found for linked user {linked_user}.")
            
            if metadata_cid:
                logger.info(f"Processing metadata CID: {metadata_cid}")
                file_metadata = download_json_from_ipfs(metadata_cid)
                file_metadata_json = download_file(file_metadata, download_path, project_id, file_cid)
                logger.info(f"Downloaded file metadata: {metadata_cid}")
                logger.info(f"file metadata: {file_metadata}")
                logger.info(file_metadata_json)
            
        
        else:
            logger.warning(f"Invalid File CID for Project ID: {project_id}. Skipping metadata processing.")


2025-01-11 22:50:04.090 | INFO     | super_helper:search_index:207 - Starting keyword search...
2025-01-11 22:50:04.090 | INFO     | super_helper:search_index:208 - Keyword: 'must'
2025-01-11 22:50:04.118 | INFO     | super_helper:search_index:219 - Search successful: Found 3 result(s).
2025-01-11 22:50:04.120 | INFO     | super_helper:search_index:221 - 1. Project Id: 73243@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmNjaCTeNdmrJfLYUpD8qhpjndXXX7QrWagRMhNSAtQkC6, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-11 22:50:04.122 | INFO     | super_helper:search_index:221 - 2. Project Id: 73243@test, File CID: QmUq29KRwpTdvScB5oYzEDobDHyb4N1f9eaZXm4VaMCgiW, Metadata CID: QmdyAE132FSeSQaqUHzE5GtMrEwy821ScAYKEXsxpWSnV5, Title: covid19-mlsf: a multi-task learning-based stock market forecasting framework during the covid-19 pandemic
2025-01-11 22:50:04.123 | INFO     | super_helper:sear